# torch pth 到onnx

https://www.lanqiao.cn/courses/40981/learning/?id=2813920&compatibility=false

---
# 重点
```python
def onnx_model_transfer(input, input_model_path, output_model_path):
    resnet_model = models.resnet18()
    
    state_dict = torch.load(input_model_path)
    resnet_model.load_state_dict(state_dict=state_dict)

    resnet_model.eval()
    torch.onnx.export(resnet_model, input, output_model_path, verbose=True)
```
* resnet18 是根据原有代码的推理 
* 定义model 、state_dict=torch.load() 加载torchmodel_path的权重、使用model加载各种权重信息，model.load_state_dict(state_dict = state_dict) 、切换为评估模式 model.eval() 因为onnx转换其实都是评估格式，没有train能力、torch.onnx.export(model,input,output_model_path) verbose是一个控制结构输出的变量
---
# 参考答案
```python
import onnx
import onnxruntime
import torch
import torchvision
import torchvision.models as models
import torch.utils.benchmark as benchmark

def onnx_model_transfer(input, input_model_path, output_model_path):
    resnet_model = models.resnet18()
    
    state_dict = torch.load(input_model_path)
    resnet_model.load_state_dict(state_dict=state_dict)

    resnet_model.eval()
    torch.onnx.export(resnet_model, input, output_model_path, verbose=True)

def resnet_predict_time(input, model_path, n):
    # 加载 ResNet-18 模型
    resnet_model = models.resnet18()
    resnet_model.load_state_dict(torch.load(model_path))
    resnet_model.eval()
    resnet_bm = benchmark.Timer(stmt='resnet_model(inputs)', globals={'resnet_model': resnet_model, 'inputs': input})
    # 预测 n 次的平均时间
    resnet_time = resnet_bm.timeit(n).mean
    return resnet_time

## ！！！没理解
def onnx_predict_time(input, model_path, n):

    ort_session = onnxruntime.InferenceSession(model_path)
    onnx_inputs = {ort_session.get_inputs()[0].name: input.numpy()}
    onnx_bm = benchmark.Timer(stmt='ort_session.run(None, onnx_inputs)', globals={'ort_session': ort_session, 'onnx_inputs': onnx_inputs})
    onnx_time = onnx_bm.timeit(n).mean
    return onnx_time

def main():
    source_model_path = 'resnet18-f37072fd.pth'
    onnx_model_path = "resnet18.onnx"
    input = torch.randn(1, 3, 224, 224)
    onnx_model_transfer(input, source_model_path, onnx_model_path)
    resnet_time = resnet_predict_time(input, source_model_path, 20)
    print('原始模型推理时间：', resnet_time)
    onnx_time = onnx_predict_time(input, onnx_model_path, 20)
    print('ONNX 模型推理时间：', onnx_time)
if __name__ == "__main__":
    main()
```